# REMDのOpenMM実装
- ChatGPTを使用
- 一定時間ごとに、レプリカ間の座標を交換する(温度はそのまま)
- 速度は交換しない
- 交換確率が0.25になるよう温度を調整するための予備MD

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import numpy as np
from tqdm.notebook import tqdm
import random
import os
from datetime import datetime

In [2]:
# 現在の日付と時刻を取得
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# ディレクトリ名を作成
output_dir = f"./output/output_{current_time}"
# ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)
print(f"ディレクトリ '{output_dir}' を作成しました！")

ディレクトリ './output/output_2024-12-06_18-41-07' を作成しました！


In [3]:
# パラメータ設定
params = {}
params['n_replicas'] = 8  # レプリカ数
replicas = [str(i) for i in np.arange(params['n_replicas'])]
print('Replicas:', replicas)
params['temperature_min'] = 300 # K
params['temperature_max'] = 500 # K
params['temperatures'] = np.linspace(params['temperature_min'], params['temperature_max'], params['n_replicas'])  # 温度 (Kelvin)
params['temperatures'] = params['temperatures'].tolist()
print('Temperatures (K):', params['temperatures'])
params['acceptance_ratio'] = 0.25

params['dt'] = 0.002 # タイムステップ(ps)
params['n_steps'] = 20000 # シミュレーション総ステップ数
params['n_steps_exchange'] = 1000 # 交換を試みる間隔 (steps)
params['n_steps_save'] = 100
params['n_steps_equil'] = 20000

params['nonbondedCutoff'] = 1.0 # nm
params['friction'] = 1.0 # /ps
params['restraint_force'] = 10 # kcal/mol/A^2

params['pdb_path'] = './structures/ala2_solvated.pdb'
params['ff'] = ['amber99sbildn.xml', 'tip3p.xml']

Replicas: ['0', '1', '2', '3', '4', '5', '6', '7']
Temperatures (K): [300.0, 328.57142857142856, 357.14285714285717, 385.7142857142857, 414.2857142857143, 442.8571428571429, 471.42857142857144, 500.0]


In [4]:
# 初期パラメータ保存
import json
filepath = f"{output_dir}/params.json"
with open(filepath, mode="wt", encoding="utf-8") as f:
	json.dump(params, f, ensure_ascii=False, indent=2)

## レプリカ交換関数の定義

In [5]:
kB = BOLTZMANN_CONSTANT_kB.value_in_unit(kilojoule/kelvin)
print(kB)

1.3806490000000003e-26


In [6]:
# エネルギー交換関数
def attempt_exchange(replica1, replica2):
    E1 = simulations[replica1].context.getState(getEnergy=True).getPotentialEnergy()
    E2 = simulations[replica2].context.getState(getEnergy=True).getPotentialEnergy()
    beta1 = 1 / (BOLTZMANN_CONSTANT_kB * params['temperatures'][replica1] * kelvin)
    beta2 = 1 / (BOLTZMANN_CONSTANT_kB * params['temperatures'][replica2] * kelvin)
    delta = (beta2 - beta1) * (E1 - E2) / AVOGADRO_CONSTANT_NA
    print('Before:\t', np.exp(-delta))
    
    new_beta2 = -np.log(params['acceptance_ratio']) / ((E1 - E2) / AVOGADRO_CONSTANT_NA) + beta1
    new_temp2 = (1 / (BOLTZMANN_CONSTANT_kB * new_beta2)).value_in_unit(kelvin)
    print(f'Changed:\t', params['temperatures'][replica2], '->' ,new_temp2)
    params['temperatures'][replica2] = new_temp2
    beta2 = 1 / (BOLTZMANN_CONSTANT_kB * params['temperatures'][replica2] * kelvin)
    delta = (beta2 - beta1) * (E1 - E2) / AVOGADRO_CONSTANT_NA
    print('After:\t', np.exp(-delta))
    print('==========')
    
    if delta < 0 or random.uniform(0, 1) < np.exp(-delta):
        # 交換を行う
        # print(f"Exchange accepted between replica {replica1} and {replica2}")
        # temp1 = temperatures[replica1]
        # temperatures[replica1] = temperatures[replica2]
        # temperatures[replica2] = temp1
        rep1 = replicas[replica1]
        replicas[replica1] = replicas[replica2]
        replicas[replica2] = rep1
        # Write log
        with open(f'{output_dir}/acceptance.csv', 'a') as f:
            # f.write(','.join(replicas)+'\n')
            f.write(f"{replica1},{replica2},accept\n")
            
        positions1 = simulations[replica1].context.getState(getPositions=True).getPositions()
        positions2 = simulations[replica2].context.getState(getPositions=True).getPositions()
        simulations[replica1].context.setPositions(positions2)
        simulations[replica2].context.setPositions(positions1)
    else:
        # print(f"Exchange rejected between replica {replica1} and {replica2}")
        # Write log
        with open(f'{output_dir}/acceptance.csv', 'a') as f:
            # f.write(','.join(replicas)+'\n')
            f.write(f"{replica1},{replica2},reject\n")
        pass

## システムの作成

In [7]:
pdb = app.PDBFile(params['pdb_path'])
forcefield = app.ForceField(*params['ff'])

In [8]:
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=params['nonbondedCutoff']*nanometer,
    constraints=app.HBonds,
)
print('System created...')

# レプリカごとに異なるシミュレーションをセットアップ
integrators = []
simulations = []
for i, temp in enumerate(params['temperatures']):
    integrator = LangevinIntegrator(
        temp*kelvin,       # 温度
        params['friction']/picosecond,    # 摩擦係数
        params['dt']*picoseconds  # タイムステップ
    )
    integrators.append(integrator)
    simulation = app.Simulation(pdb.topology, system, integrator)
    # simulation.context.setPositions(pdb.positions)
    # simulation.context.setVelocitiesToTemperature(temp*kelvin)
    simulations.append(simulation)

System created...


## 平衡化

In [9]:
%%time
system_equil = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=params['nonbondedCutoff']*nanometer,
    constraints=app.HBonds,
)
# Positional restraints for all heavy-atoms for equilibration
pos_res = CustomExternalForce("k*periodicdistance(x, y, z, x0, y0, z0)^2;")
pos_res.addPerParticleParameter("k")
pos_res.addPerParticleParameter("x0")
pos_res.addPerParticleParameter("y0")
pos_res.addPerParticleParameter("z0")

for ai, atom in enumerate(pdb.topology.atoms()):
    if atom.name == 'CA':
        x = pdb.positions[ai][0].value_in_unit(nanometers)
        y = pdb.positions[ai][1].value_in_unit(nanometers)
        z = pdb.positions[ai][2].value_in_unit(nanometers)
        pos_res.addParticle(ai, [params['restraint_force']*kilocalories_per_mole/(angstrom**2), x, y, z])

system_equil.addForce(pos_res)
print('System_equil created...')

integrators_equil = []
simulations_equil = []
for i, temp in enumerate(params['temperatures']):
    integrator = LangevinIntegrator(
        temp*kelvin,       # 温度
        params['friction'],    # 摩擦係数
        params['dt']*picoseconds  # タイムステップ
    )
    # integrators_equil.append(integrator)
    simulation_equil = app.Simulation(pdb.topology, system_equil, integrator)
    simulation_equil.context.setPositions(pdb.positions)
    simulation_equil.minimizeEnergy()
    simulation_equil.context.setVelocitiesToTemperature(temp*kelvin)
    print("Equilibrating for", params['n_steps_equil'], "steps")
    simulation_equil.step(params['n_steps_equil'])
    # simulations_equil.append(simulation)
    simulations[i].context.setPositions(
        simulation_equil.context.getState(getPositions=True ).getPositions()
    )
    simulations[i].context.setVelocities(
        simulation_equil.context.getState(getVelocities=True).getVelocities()
    )

System_equil created...
Equilibrating for 20000 steps
Equilibrating for 20000 steps
Equilibrating for 20000 steps
Equilibrating for 20000 steps
Equilibrating for 20000 steps
Equilibrating for 20000 steps
Equilibrating for 20000 steps
Equilibrating for 20000 steps
CPU times: user 19.4 s, sys: 3.51 s, total: 22.9 s
Wall time: 23.5 s


## レポーターの追加

In [10]:
# append reporters
print('Adding Reporters...')
dcd_reporters = [app.DCDReporter(file=f'{output_dir}/replica_{i}.dcd', reportInterval=params['n_steps_save']) 
                 for i, temp in enumerate(params['temperatures'])]
state_data_reporters = [app.StateDataReporter(file=f'{output_dir}/replica_{i}.log', 
                                              reportInterval=params['n_steps_save'], 
                                              totalSteps=params['n_steps'], 
                                              step=True,
                                              potentialEnergy=True,
                                              temperature=True,
                                              speed=True, 
                                              progress=True, 
                                              elapsedTime=True) 
                        for i, temp in enumerate(params['temperatures'])]

# 各レプリカにreporterを追加
for i, temp in enumerate(params['temperatures']):
    simulations[i].reporters.append(dcd_reporters[i])
    simulations[i].reporters.append(state_data_reporters[i])

Adding Reporters...


## Production
- 隣接ペアの組み合わせを1回毎に変えるようにしたい
- accept/rejectのログを記録したい

In [11]:
for i in range(1, 8-1, 2):
    print(i, i+1)

1 2
3 4
5 6


In [12]:
%%time
# シミュレーションループ
print('Production...')
is_even_step = True # 交換ペアを交互に変えるための変数
for step in tqdm(range(0, params['n_steps'], params['n_steps_exchange']), leave=False):
    # print(f'*** Step {step} ***"')
    for sim in simulations:
        sim.step(params['n_steps_exchange'])  # 各レプリカで実行

    # レプリカ間での交換試行
    if is_even_step:
        for i in range(0, params['n_replicas'], 2):
            attempt_exchange(i, i + 1)
    else:
        for i in range(1, params['n_replicas']-1, 2):
            attempt_exchange(i, i + 1)
            
    is_even_step = not is_even_step
        
    with open(f'{output_dir}/replicas.csv', 'a') as f:
            f.write(','.join(replicas)+'\n')

Production...


  0%|          | 0/20 [00:00<?, ?it/s]

Before:	 3.360529773082805e-08
Changed:	 328.57142857142856 -> 302.11634639083576
After:	 0.2499999999999992
Before:	 0.1233548299339372
Changed:	 385.7142857142857 -> 375.57221425575847
After:	 0.25000000000000033
Before:	 0.0028756206830128704
Changed:	 442.8571428571429 -> 420.71624497262945
After:	 0.2499999999999992
Before:	 0.001956470703519057
Changed:	 500.0 -> 477.4936468191158
After:	 0.24999999999999598
Before:	 5.038367173298688e-07
Changed:	 357.14285714285717 -> 306.632886111762
After:	 0.24999999999999808
Before:	 7.328101730967849e-05
Changed:	 414.2857142857143 -> 380.75268002447643
After:	 0.25000000000000167
Before:	 0.0027767332590586066
Changed:	 471.42857142857144 -> 431.6515451462119
After:	 0.25000000000000105
Before:	 0.865149344902462
Changed:	 302.11634639083576 -> 321.55751797255124
After:	 0.25000000000000067
Before:	 1.571891893839954e-15
Changed:	 375.57221425575847 -> 308.93920688500964
After:	 0.25000000000000383
Before:	 1.3484288830667045e-05
Changed:

## 最終構造を保存

In [13]:
# シミュレーション結果を保存
for i, sim in enumerate(simulations):
    state = sim.context.getState(getPositions=True, getEnergy=True)
    with open(f"{output_dir}/output_replica_{i}.pdb", "w") as f:
        app.PDBFile.writeFile(pdb.topology, state.getPositions(), f)

In [14]:
print(params['temperatures'])

[300.0, 302.26014322042374, 304.1734945694696, 308.61391545385584, 311.4241021193732, 380.039117007804, 392.7033063077621, 359.3622883335206]
